# Quick dataset analysis tutorial - fastdup V1.0

In [ ]:
# install fastdup
!pip install pip -U
!pip install fastdup

In [1]:
# download oxford pet images
!wget https://thor.robots.ox.ac.uk/~vgg/data/pets/images.tar.gz -O images.tar.gz
!tar xf images.tar.gz

--2023-03-01 06:49:49--  https://thor.robots.ox.ac.uk/~vgg/data/pets/images.tar.gz
Resolving thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)... 129.67.95.98
Connecting to thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)|129.67.95.98|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 791918971 (755M) [application/octet-stream]
Saving to: ‘images.tar.gz’

images.tar.gz       100%[===================>] 755.23M  21.7MB/s    in 37s     

2023-03-01 06:50:27 (20.5 MB/s) - ‘images.tar.gz’ saved [791918971/791918971]



# 1. Import fastdup

In [4]:
import fastdup

# 2. Run fastdup on the pet images

In [6]:
images_dir = 'images/'
work_dir = 'images_unlabeled/'

In [7]:
# create and run fastdup
fd = fastdup.create(work_dir=work_dir, input_dir=images_dir)
fd.run(ccthreshold=0.9, threshold=0.8)

FastDup Software, (C) copyright 2022 Dr. Amir Alush and Dr. Danny Bickson.
fastdup C++ error received:  read_features_parallel allowed values 1..64
 



NoneType: None
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/fastdup/sentry.py", line 114, in inner_function
    ret = func(*args, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/fastdup/fastdup_controller.py", line 300, in run
    self._create_img_mapping()
  File "/usr/local/lib/python3.8/dist-packages/fastdup/fastdup_controller.py", line 617, in _create_img_mapping
    df_mapping = self._fetch_df(FD.MAPPING_CSV).reset_index()
AttributeError: 'NoneType' object has no attribute 'reset_index'


AttributeError: ignored

# 3. Finding clusters of similar images

In [ ]:
# visualize clusters by size
fd.vis.component_gallery(sort_by='comp_size', max_width=700)

In [ ]:
# visualize clusters by similarity
fd.vis.component_gallery(sort_by='distance', max_width=700, min_items=2, ascending=False)

In [ ]:
# get connected components
cc_df, _ = fd.connected_components()
cc_df[cc_df.mean_distance > 0.99]
cc_df.sort_values(by=['count'], ascending=False).head(3)

,fastdup_id,component_id,sum,count,mean_distance,min_distance,max_distance,img_filename,error_code,is_valid
6316,6322,5873,22.1163,24.0,0.9215,0.9012,0.9407,samoyed_29.jpg,VALID,True
6342,6348,5873,22.1163,24.0,0.9215,0.9012,0.9407,samoyed_52.jpg,VALID,True
6344,6350,5873,22.1163,24.0,0.9215,0.9012,0.9407,samoyed_54.jpg,VALID,True


In [ ]:
# analyze connected components
agg_dict = {'img_filename': list, 'mean_distance': max}
duplicates_df = cc_df.groupby('component_id').agg(agg_dict)
duplicates_df = duplicates_df.sort_values(by=['mean_distance'], ascending=False)
duplicates_df.head(5)

,img_filename,mean_distance
component_id,,
5171,"[newfoundland_137.jpg, newfoundland_153.jpg]",1.0
605,"[Bombay_118.jpg, Bombay_209.jpg]",1.0
609,[Bombay_121.jpg],1.0
613,[Bombay_126.jpg],1.0
4599,"[keeshond_103.jpg, keeshond_167.jpg]",1.0


# 4. Finding outlier images in the data

In [ ]:
# visualize outlier images
fd.vis.outliers_gallery()

## 4.b. Getting a detailed list

In [ ]:
# show stats for furthest outliers
fd.outliers().sort_values(by=['distance']).head(5)

,index,outlier,nearest,distance,img_filename_outlier,error_code_outlier,is_valid_outlier,img_filename_nearest,error_code_nearest,is_valid_nearest
0,737,207,4127,0.596920,Bengal_105.jpg,VALID,True,german_shorthaired_33.jpg,VALID,True
1,735,236,1160,0.611524,Bengal_131.jpg,VALID,True,Egyptian_Mau_6.jpg,VALID,True
2,734,6937,3517,0.617133,staffordshire_bull_terrier_51.jpg,VALID,True,chihuahua_24.jpg,VALID,True
3,733,5374,6631,0.621796,miniature_pinscher_76.jpg,VALID,True,shiba_inu_128.jpg,VALID,True
4,732,2223,2231,0.622756,Sphynx_128.jpg,VALID,True,Sphynx_137.jpg,VALID,True


# 5. Visualize blurry, dark and bright images

In [ ]:
# visualize blurry images
fd.vis.stats_gallery(metric='blur')

In [ ]:
# visualize dark images
fd.vis.stats_gallery(metric='dark')

In [ ]:
# visualize bright images
fd.vis.stats_gallery(metric='bright')

In [ ]:
# show image statistics
fd.img_stats().head(5)

,fastdup_id,img_w,img_h,unique,blur,mean,min,max,stdv,file_size,contrast,img_filename,error_code,is_valid
0,0,600,400,0,586.0380,69.5999,0.0,242.0,37.9883,32362,1.0,Abyssinian_1.jpg,VALID,True
1,1,375,500,0,808.7275,69.1038,0.0,249.0,52.6960,96294,1.0,Abyssinian_10.jpg,VALID,True
2,2,394,500,0,4120.0146,142.0682,0.0,255.0,84.0555,126923,1.0,Abyssinian_100.jpg,VALID,True
3,3,450,313,0,2440.4067,128.6538,0.0,255.0,78.9571,49023,1.0,Abyssinian_101.jpg,VALID,True
4,4,500,465,0,4965.8892,127.4869,0.0,255.0,57.4010,165505,1.0,Abyssinian_102.jpg,VALID,True
